# Datasets

Run this `gcsfuse` cell if you can't list the folders inside of "/gcs"

In [7]:
!gcsfuse --implicit-dirs "~/gcs"

I1110 03:35:00.800448 2023/11/10 03:35:00.800416 Start gcsfuse/0.42.5 (Go version go1.20.3) for app "" using mount point: /home/jupyter/gcs
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithConn: Mount: mount: running /usr/bin/fusermount: exit status 1


When using GCS buckets, use "/gcs" instead of "gs://"

In [8]:
#from sklearn.model_selection import train_test_split

dataset_path = "/home/jupyter/gcs/serena-shsw-datasets"

training_dataset = dataset_path + "/FER-2013/train"
training_labels = [] # Fill this!

test_dataset = dataset_path + "/FER-2013/test"
test_labels = [] # Fill this!

validation_dataset = dataset_path + "/FER-2013/valid"

saved_model_path = dataset_path + "/models/serena-emotion-detector/model"

# Split the dataset into training and validation sets
#training_dataset, validation_dataset = train_test_split(training_dataset, test_size=0.2, random_state=42)

# Output directory contents
!echo "Train"
!ls {training_dataset}
!echo "Test"
!ls {test_dataset}
!echo "Valid"
!ls {validation_dataset}

Train
angry  disgust	fear  happy  neutral  sad  surprise
Test
angry  disgust	fear  happy  neutral  sad  surprise
Valid


# Import Library

In [9]:
import csv
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Evaluate Model

In [10]:
saved_emotion_detector = tf.keras.models.load_model(saved_model_path)

# Check its architecture
saved_emotion_detector.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 14,847,815
Trainable params: 133,127
Non-trainable params: 14,714,688
______________________________________

Check accuracy of saved_emotion_detector

In [11]:
val_loss, val_accuracy = saved_emotion_detector.evaluate(training_dataset, training_labels)
print(f'Validation Accuracy: {val_accuracy}')

test_loss, test_accuracy = saved_emotion_detector.evaluate(test_data, test_labels)
print(f'Test Accuracy: {test_accuracy}')

ValueError: Failed to find data adapter that can handle input: <class 'str'>, (<class 'list'> containing values of types set())